__Importing important libraries__

In [1]:
# Import necessary libraries
import numpy as np
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.primitives import Estimator
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import ZFeatureMap

# For machine learning components
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityQuantumKernel as QuantumKernel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

print("Required libraries imported successfully.")

Required libraries imported successfully.


 __VQE for Feature Generation (Molecular Ground State Energy)__

In [2]:
# --- 1. VQE for Feature Generation (Molecular Ground State Energy) ---

def get_simplified_h2_qubit_hamiltonian(param_value):
    """Constructs a simplified effective H2 qubit Hamiltonian."""
    coeffs = [
        0.5,                  # II
        -0.2 * param_value,   # IZ
        0.3,                  # ZI
        0.1 * param_value,    # ZZ
        0.2                   # XX
    ]
    paulis = ['II', 'IZ', 'ZI', 'ZZ', 'XX']
    return SparsePauliOp(paulis, coeffs)

def run_vqe_for_molecule(qubit_op):
    """Runs VQE to find the ground state energy."""
    ansatz = ZFeatureMap(qubit_op.num_qubits, reps=1)
    optimizer = COBYLA(maxiter=100)
    vqe = VQE(Estimator(), ansatz, optimizer)
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    return result.eigenvalue.real

In [3]:
# --- 2. Create Dataset ---
param_values_class0 = np.linspace(0.8, 1.0, 10)
param_values_class1 = np.linspace(1.2, 1.4, 10)

features, labels = [], []
print("\nGenerating dataset from VQE energies...")

for param_val in param_values_class0:
    qubit_op = get_simplified_h2_qubit_hamiltonian(param_val)
    energy = run_vqe_for_molecule(qubit_op)
    features.append([energy])
    labels.append(0)

for param_val in param_values_class1:
    qubit_op = get_simplified_h2_qubit_hamiltonian(param_val)
    energy = run_vqe_for_molecule(qubit_op)
    features.append([energy])
    labels.append(1)

X = np.array(features)
y = np.array(labels)



Generating dataset from VQE energies...


C:\Users\asmit\AppData\Local\Temp\ipykernel_7156\2922919438.py:19: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(Estimator(), ansatz, optimizer)
C:\Users\asmit\AppData\Local\Temp\ipykernel_7156\2922919438.py:19: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(Estimator(), ansatz, optimizer)
C:\Users\asmit\AppData\Local\Temp\ipykernel_7156\2922919438.py:19: D

__Preparing Data__

In [4]:
# --- 3. Prepare Data ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

__Model Tranning__

1. Quantum Models: QSVC

In [5]:
# --- 4. Quantum Models: QSVC ---
print("\n--- Training Quantum SVM (QSVC) ---")
feature_map = ZFeatureMap(feature_dimension=X_train_scaled.shape[1], reps=2)
kernel = QuantumKernel(feature_map=feature_map)
qsvc_model = QSVC(quantum_kernel=kernel)
qsvc_model.fit(X_train_scaled, y_train)
y_pred_qsvc = qsvc_model.predict(X_test_scaled)
accuracy_qsvc = accuracy_score(y_test, y_pred_qsvc)
print(f"QSVC Accuracy: {accuracy_qsvc * 100:.2f}%")


--- Training Quantum SVM (QSVC) ---
QSVC Accuracy: 66.67%


2. Classical Models

In [6]:
# --- 5. Classical Models ---
print("\n--- Training Classical SVM ---")
classical_svm = SVC(kernel='rbf', random_state=42)
classical_svm.fit(X_train_scaled, y_train)
y_pred_svm = classical_svm.predict(X_test_scaled)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"Classical SVM Accuracy: {accuracy_svm * 100:.2f}%")


--- Training Classical SVM ---
Classical SVM Accuracy: 33.33%


3. Training Random Forest

In [7]:
print("\n--- Training Random Forest ---")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")


--- Training Random Forest ---
Random Forest Accuracy: 66.67%


__Comparing the MOdel Perfromance__

In [8]:

# --- 6. Results ---
print("\n--- Model Performance Comparison ---")
print(f"QSVC Accuracy:          {accuracy_qsvc * 100:.2f}%")
print(f"Classical SVM Accuracy: {accuracy_svm * 100:.2f}%")
print(f"Random Forest Accuracy: {accuracy_rf * 100:.2f}%")


--- Model Performance Comparison ---
QSVC Accuracy:          66.67%
Classical SVM Accuracy: 33.33%
Random Forest Accuracy: 66.67%
